In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df_train = pd.read_csv('../input/playground-series-s3e22/train.csv')
df_test = pd.read_csv('../input/playground-series-s3e22/test.csv')

In [ ]:
y = df_train['outcome']

In [ ]:
df_train = df_train.drop('outcome', axis=1)
df_full = pd.concat([df_train, df_test])

In [ ]:
df_full

In [ ]:
df_train.head()

In [ ]:
df_test.head()

# Data Analysis #

In [ ]:
y

In [ ]:
sns.countplot(data=y, x=y.index)

## Data Preprocessing ##

In [ ]:
df_full.columns

In [ ]:
null_df = pd.Series(df_full.isnull().sum())

In [ ]:
sns.barplot(x=null_df.sort_values().index,y=null_df.sort_values())
plt.xticks(rotation=90);
plt.ylabel('Missing values')

In [ ]:
percentage_missing_data = (null_df.sort_values()/df_full.shape[0])*100
percentage_missing_data

In [ ]:
df_full.shape

Dropping the columns with more than 10% of missing information. Later on we will drop the rows for those features with less than 1.8% of missing information.

In [ ]:
df_full = df_full.drop(['abdomen', 'rectal_exam_feces'], axis=1)

In [ ]:
percentage_missing_data = (df_full.isnull().sum().sort_values()/df_full.shape[0])*100

In [ ]:
for i in list(percentage_missing_data[percentage_missing_data != 0].index):
    uniques_list = df_full[i].unique()
    print(f'{i} has uniques {uniques_list}' )
    

In [ ]:
corr_df = df_full.copy()

In [ ]:
corr_df.capillary_refill_time = corr_df.capillary_refill_time.replace({'more_3_sec':1, 'less_3_sec':-1, '3':0})
corr_df.peristalsis = corr_df.peristalsis.replace({'absent':0, 'hypomotile':-1, 'normal':1, 'hypermotile':2, 'distend_small':0.5})
corr_df.mucous_membrane = corr_df.mucous_membrane.replace({'dark_cyanotic':1, 'pale_cyanotic':2, 'pale_pink':3, 'normal_pink':4, 'bright_pink':5,'bright_red':6})
corr_df.nasogastric_reflux = corr_df.nasogastric_reflux.replace({'less_1_liter':-1, 'more_1_liter':2, 'none':0, 'slight':1})
corr_df.abdominal_distention = corr_df.abdominal_distention.replace({'slight':0.5, 'moderate':1, 'none':0, 'severe':2})
corr_df.temp_of_extremities  = corr_df.temp_of_extremities.replace({'cool':0.5, 'cold':-1, 'normal':1, 'warm':2})
corr_df.pain = corr_df.pain.replace({'depressed':1, 'mild_pain':2, 'extreme_pain':3, 'alert':4, 'severe_pain':5, 'slight':6, 'moderate':7})
corr_df.abdomo_appearance = corr_df.abdomo_appearance.replace({'serosanguious':1, 'cloudy':0.5, 'clear':0})
corr_df.peripheral_pulse = corr_df.peripheral_pulse.replace({'reduced':0.5, 'normal':1,  'absent':0, 'increased':2})
corr_df.nasogastric_tube = corr_df.nasogastric_tube.replace({'slight':1, 'none':0, 'significant':2})

In [ ]:
corr_df = corr_df.select_dtypes(include=['float64','int64']).corr()

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(corr_df, annot=False)

In [ ]:
corr_df[list(percentage_missing_data[percentage_missing_data != 0].index)]

After this exploration in search of correlations in between complete features and those with missing data we saw that there are non strong correlation, just moderate ones. Thus we will proceed with different methods of substitution.

In [ ]:
df_full[['pulse','peripheral_pulse', 'capillary_refill_time']]

In [ ]:
percentage_missing_data

*Feature: abdominal_distention*

In [ ]:
df_full.abdominal_distention.value_counts()

In [ ]:
# Following the filling method of bfill: use next valid observation to fill gap.
df_full.abdominal_distention = df_full.abdominal_distention.fillna(method='bfill',axis=0)

*Feature: temp_of_extremities*

In [ ]:
df_full.temp_of_extremities.value_counts()

In [ ]:
#We will fill with the max, since the statistic shows a incline to those values
df_full.temp_of_extremities = df_full.temp_of_extremities.fillna('cool')

*Feature: pain*

In [ ]:
df_full.pain.value_counts()

In [ ]:
df_full.pain = df_full.pain.fillna(method='bfill',axis=0)

*Feature: abdomo_appearance*

In [ ]:
df_full.abdomo_appearance.value_counts()

In [ ]:
df_full.abdomo_appearance = df_full.abdomo_appearance.fillna(method='bfill',axis=0)

*Feature: peripheral_pulse*

In [ ]:
def my_rand(p):
    return np.random.rand() > p

In [ ]:
df_full.peripheral_pulse.value_counts()

In [ ]:
def choice(x):
    if x in ['reduced', 'normal', 'absent', 'increased'] :
        return x
    else:
        p = my_rand((714*100)/1916)
        if p == True:
            return 'reduced'
        else:
            return 'normal'

In [ ]:
# Since the mayority lies in reduced and normal, we will randomly put the missing values on the high statistic, at the end we expect not great difference on the choice of reduce and normal since they are close to each other
df_full.peripheral_pulse = df_full.apply(lambda x: choice(x.peripheral_pulse), axis=1)

*Feature: nasogastric_tube*

In [ ]:
#Missing 80 values, we will assume that the NaN are none answers since the person didn't saw that feature as important
df_full.nasogastric_tube.value_counts()

In [ ]:
df_full.nasogastric_tube = df_full.nasogastric_tube.fillna('slight')

*Feature:peristalsis* 

In [ ]:
df_full.peristalsis.value_counts()

In [ ]:
def choice2(x):
    if x in ['absent', 'hypomotile', 'normal', 'hypermotile','distend_small'] :
        return x
    else:
        p = my_rand((741*100)/(1111+741))
        if p == True:
            return 'hypomotile'
        else:
            return 'absent'

In [ ]:
df_full.peristalsis = df_full.apply(lambda x: choice2(x.peristalsis), axis=1)

*Feature: nasogastric_reflux*

In [ ]:
df_full.nasogastric_reflux.value_counts()

In [ ]:
df_full.nasogastric_reflux = df_full.nasogastric_reflux.fillna(method='bfill',axis=0)

*Feature: mucous_membrane*

In [ ]:
df_full.mucous_membrane.value_counts()

In [ ]:
df_full.mucous_membrane = df_full.mucous_membrane.fillna(method='bfill',axis=0)

*Feature: capillary_refill_time*

In [ ]:
df_full.capillary_refill_time.value_counts()

In [ ]:
df_full.capillary_refill_time = df_full.capillary_refill_time.fillna('less_3_sec')
#df_full.capillary_refill_time = df_full.capillary_refill_time.fillna(method='bfill',axis=0)

*Feature: abdomo_appearance*

In [ ]:
df_full.abdomo_appearance.value_counts()

In [ ]:
df_full.abdomo_appearance = df_full.abdomo_appearance.fillna(method='ffill',axis=0)

In [ ]:
df_full.shape

In [ ]:
percentage_missing_data = (df_full.isnull().sum().sort_values()/df_full.shape[0])*100
percentage_missing_data

## One-hot encoding ##

In [ ]:
df_full.select_dtypes(include='object').columns

In [ ]:
dummies_df = pd.get_dummies(df_full[['surgery', 'age', 'temp_of_extremities', 'peripheral_pulse',
       'mucous_membrane', 'capillary_refill_time', 'pain', 'peristalsis',
       'abdominal_distention', 'nasogastric_tube', 'nasogastric_reflux',
       'abdomo_appearance', 'surgical_lesion', 'cp_data']], drop_first=True, dtype=int)

In [ ]:
df_full_final = df_full.drop(['surgery', 'age', 'temp_of_extremities', 'peripheral_pulse',
       'mucous_membrane', 'capillary_refill_time', 'pain', 'peristalsis',
       'abdominal_distention', 'nasogastric_tube', 'nasogastric_reflux',
       'abdomo_appearance', 'surgical_lesion', 'cp_data'],axis=1)

In [ ]:
df_full_final = pd.concat([df_full_final, dummies_df], axis=1)

In [ ]:
df_full_final.shape

In [ ]:
df_full_final.info()

## Train Test Split ##

In [ ]:
X = df_full_final[df_full_final.id <= 1234]
X_test = df_full_final[df_full_final.id > 1234]

In [ ]:
X.shape

In [ ]:
X_test.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#erased_values = [ele for ele in range(max(X_train.id.values)+1) if ele not in X_train.id.values]

In [ ]:
#y_train = y_train.drop(erased_values)

In [ ]:
y.value_counts()

In [ ]:
y.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder  

le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
le.inverse_transform([0,1,2])

In [ ]:
le.inverse_transform(y);

In [ ]:
y.shape

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

## Normalizing Data ##

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
#X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)

# Creating the model #

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import SGD, Adam

In [ ]:
[2**i for i in range(0,15)]

In [ ]:
def create_model():
    model = Sequential()

    model.add(Dense(49, activation='relu', input_shape=[49]))
    #model.add(BatchNormalization(synchronized=True))
    model.add(Dense(25, activation='relu',kernel_regularizer=regularizers.L1L2(l1=0.05, l2=0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(13, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(8, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1,activation='relu'))
    model.compile(optimizer=Adam(lr=0.01), loss='mse', metrics=['accuracy'])
    return model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', patience=20, verbose=0)

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score

#for train_index, valid_index in kfold.split(X_scaled):
#    X_train, X_valid = X_scaled[train_index], X_scaled[valid_index]
#    y_train, y_valid = y[train_index], y[valid_index]
    
#    model = create_model()
#    model.fit(X_train, y_train,
#         validation_data=[X_valid,y_valid],
#         epochs=100,
#         verbose=0,
#         batch_size = 32,
#         callbacks=[early_stop])
    
#    y_pred = model.predict(X_valid).astype('int32')
    
#    accuracy = accuracy_score(y_valid, y_pred)
#    print(f'Accuracy: {accuracy}')
    

In [ ]:
model = create_model()
model.fit(X_train, y_train,
         validation_data=[X_valid,y_valid],
         epochs=100,
         verbose=1,
         batch_size = 32,
         callbacks=[early_stop])

In [ ]:
pd.DataFrame(model.history.history)[['loss','val_loss']].plot()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

In [ ]:
predictions = model.predict(X_valid).astype('int32')

In [ ]:
predictions.shape

In [ ]:
predictions;

In [ ]:
matrix = confusion_matrix(y_true=y_valid, y_pred=predictions )

In [ ]:
dis = ConfusionMatrixDisplay(matrix)
dis.plot()
plt.show()

In [ ]:
print(classification_report(y_true=y_valid, y_pred=predictions))

In [ ]:
predictions_2 = model.predict(X_test_scaled).astype('int32')

In [ ]:
predictions_2.shape

In [ ]:
predictions_2;

In [ ]:
outcome_NN = le.inverse_transform(predictions_2.reshape(824,))

In [ ]:
results_NN = pd.DataFrame(index=X_test.id, data=outcome_NN, columns=['outcome'])

In [ ]:
results_NN.value_counts()

# Model Random Forrest #

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RFmodel = RandomForestClassifier()

In [ ]:
param_grid={'n_estimators':[100,200,150,300,400,500],
           'criterion':['gini', 'entropy'],
           'max_features':['log2','sqrt']}

In [ ]:
grid_model = GridSearchCV(estimator=RFmodel,
                          param_grid=param_grid,
                         scoring='neg_mean_squared_error',
                         cv=5,
                         verbose=1)

In [ ]:
grid_model.fit(X_train, y_train)

In [ ]:
grid_model.best_params_

In [ ]:
prediction_RF = grid_model.predict(X_valid)

In [ ]:
matrix = confusion_matrix(y_true=y_valid, y_pred=prediction_RF )
dis = ConfusionMatrixDisplay(matrix)
dis.plot()
plt.show()

In [ ]:
print(classification_report(y_true=y_valid, y_pred=prediction_RF))

In [ ]:
prediction_RF_result = grid_model.predict(X_test_scaled)

In [ ]:
outcome = le.inverse_transform(prediction_RF_result.reshape(824,))
results_RF = pd.DataFrame(index=X_test.id, data=outcome, columns=['outcome'])
results_RF.value_counts()

# Model KNN #

In [ ]:
KNNmodel = KNeighborsClassifier()

In [ ]:
param_grid={'n_neighbors':[10, 50,100,200,150],
           'algorithm':['auto', 'ball_tree', 'kd_tree'],
           'weights':['uniform', 'distance']}

In [ ]:
grid_model_KNN = GridSearchCV(estimator=KNNmodel,
                          param_grid=param_grid,
                         scoring='neg_mean_squared_error',
                         cv=5,
                         verbose=1)

In [ ]:
grid_model_KNN.fit(X_train, y_train)

In [ ]:
grid_model_KNN.best_params_

In [ ]:
prediction_KNN = grid_model_KNN.predict(X_valid)

In [ ]:
matrix = confusion_matrix(y_true=y_valid, y_pred=prediction_KNN )
dis = ConfusionMatrixDisplay(matrix)
dis.plot()
plt.show()

In [ ]:
print(classification_report(y_true=y_valid, y_pred=prediction_KNN))

In [ ]:
prediction_KNN_result = grid_model_KNN.predict(X_test_scaled)

In [ ]:
outcome = le.inverse_transform(prediction_KNN_result.reshape(824,))
results = pd.DataFrame(index=X_test.id, data=outcome, columns=['outcome'])
results.value_counts()

# Model Gradient Boosting #

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
GBmodel = GradientBoostingClassifier()

In [ ]:
param_grid={'n_estimators':[10,100,200,150,300,400],
           'criterion':['friedman_mse', 'squared_error'],
           'max_features':['log2','sqrt'],
           'learning_rate':[0.3,1.0,1.5]}

In [ ]:
grid_model_GB = GridSearchCV(estimator=GBmodel,
                          param_grid=param_grid,
                         scoring='neg_mean_squared_error',
                         cv=5,
                         verbose=1)

In [ ]:
grid_model_GB.fit(X_train, y_train)

In [ ]:
grid_model_GB.best_params_

In [ ]:
prediction_GB = grid_model_GB.predict(X_valid)

In [ ]:
matrix = confusion_matrix(y_true=y_valid, y_pred=prediction_GB )
dis = ConfusionMatrixDisplay(matrix)
dis.plot()
plt.show()

In [ ]:
print(classification_report(y_true=y_valid, y_pred=prediction_GB))

In [ ]:
prediction_GB_result = grid_model_GB.predict(X_test_scaled)

In [ ]:
outcome = le.inverse_transform(prediction_GB_result.reshape(824,))
results = pd.DataFrame(index=X_test.id, data=outcome, columns=['outcome'])
results.value_counts()

In [ ]:
results_RF.to_csv('submission.csv')